モデルのチューニングについて

In [4]:
#共通前処理

import warnings
warnings.filterwarnings('ignore')

#必要ライブラリのインポート
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import japanize_matplotlib

#データフレーム表示用関数
from IPython.display import display

#表示オプションの調整

#浮動小数点の表示
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.4f}'.format

#データフレームのすべての項目を表示する
pd.set_option('display.max_columns', None)

#グラフのフォント
plt.rcParams['font.size'] = 14

#乱数
random_seed = 123

In [5]:
#データの読み込み

from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

#入力データx, 正解データyの作成
x = cancer.data
y = cancer.target

In [6]:
#上で読み込んだデータの分割

#分割用のパラメータ。訓練データ90%,検証データ10%の比率で分割する。
#test_size=0.1は訓練データ10%の意味
test_size = 0.1

#データ分割.訓練データと検証データ
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                   test_size=test_size, random_state=random_seed,stratify=y)

#分割後のサイズ確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

#出力で分割されていることを確認する

(569, 30)
(512, 30)
(57, 30)


In [7]:
#上で作成したデータをもとに複数のアルゴリズムリストを作成する
#複数のアルゴリズムを作成し、制度を比較する
#結果が同じになるようにrandom_stateは同じにする

#線形回帰
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

#サポートベクターマシン
from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=random_seed)

#決定木
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=random_seed)

#XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)

#アルゴリズムのリスト作成
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]


In [8]:
#for文を実行して、アルゴリズムのリストを次々に実行する
for algorithm in algorithms:
    
    #訓練データで学習
    algorithm.fit(x_train, y_train)
    
    #検証データで精度を調べる
    score = algorithm.score(x_test, y_test)
    
    #アルゴリズム名取得。__class__.__name__はおまじない
    name = algorithm.__class__.__name__
    
    #精度とアルゴリズムを表示する
    print(f'score: {score:.4f} {name}')

score: 0.9474 LogisticRegression
score: 0.8947 SVC
score: 0.9474 DecisionTreeClassifier
score: 0.9298 RandomForestClassifier
[12:07:45] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
score: 0.9825 XGBClassifier


ハイパーパラメータの最適化

In [9]:
#SVCのデフォルトパラメータの確認
algorithm = SVC(kernel='rbf', random_state=random_seed)
print(algorithm)

SVC(random_state=123)


In [10]:
#gamma値の最適化
algorithm = SVC(kernel='rbf', random_state=random_seed)
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorithm.gamma = gamma
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f} gamma:{gamma}')
    
#for文での出力結果でガンマ値を0.001を採用する

score: 0.6316 gamma:1
score: 0.6316 gamma:0.1
score: 0.6316 gamma:0.01
score: 0.9474 gamma:0.001
score: 0.9474 gamma:0.0001
score: 0.9474 gamma:1e-05


In [11]:
#Cの最適化
#gammaについては上で出した0.001をつかう

Cs = [1, 10, 100, 1000, 10000]

for C in Cs:
    algorithm = SVC(kernel='rbf', gamma=0.001, C = C, random_state=random_seed)
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f} C: {C}')

score: 0.9474 C: 1
score: 0.9298 C: 10
score: 0.9298 C: 100
score: 0.9298 C: 1000
score: 0.9298 C: 10000


交差検定法について

In [15]:
#特定のアルゴリズムで交差検定法を実施

#アルゴリズム
algorithm = SVC(kernel='rbf', random_state=random_seed,gamma=0.001, C=1)

#分割時に正解データの分布が偏らないようにstratifiedkfoldを実施
from sklearn.model_selection import StratifiedKFold
stratifiedKfold = StratifiedKFold(n_splits=3)

#交差検定法の実施
from sklearn.model_selection import cross_val_score
scores = cross_val_score(algorithm, x_train, y_train, cv=stratifiedKfold)

#平均値の計算
mean = scores.mean()

#結果
print(f'平均スコア: {mean:.4f} 個別スコア: {scores}')

平均スコア: 0.9141 個別スコア: [0.8889 0.9181 0.9353]


In [16]:
#候補アルゴリズムのリスト作成

from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=random_seed, gamma=0.001, C=1)

from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

from sklearn.ensemble import RandomForestClassifier
algorithm4 =RandomForestClassifier(random_state=random_seed)

from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)


algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]


In [17]:
#交差検定法を用いて複数のアルゴリズムの精度を比較する

#分割時に正解データの分布が偏らないようにする
from sklearn.model_selection import StratifiedKFold
stratifiedKfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
#交差検定法を実施
for algorithm in algorithms:
    scores = cross_val_score(algorithm, x_train, y_train, cv=stratifiedKfold)
    
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'平均スコア: {score:.4f} 個別スコア:{scores} {name}')
    

平均スコア: 0.9453 個別スコア:[0.9357 0.9474 0.9529] LogisticRegression
平均スコア: 0.9141 個別スコア:[0.8889 0.9181 0.9353] SVC
平均スコア: 0.9062 個別スコア:[0.8713 0.9415 0.9059] DecisionTreeClassifier
平均スコア: 0.9629 個別スコア:[0.9649 0.9591 0.9647] RandomForestClassifier
[12:59:16] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:59:16] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:59:16] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
平均ス

グリッドリサーチについて

In [19]:
#グリッドリサーチに交差検定法を組み合わせて最適なパラメータを探す

params = {
    'C':[1, 10, 100, 1000, 10000],
    'gamma':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}

algorithm =SVC(random_state=random_seed)

from sklearn.model_selection import StratifiedKFold
stratifiedKfold = StratifiedKFold(n_splits=3)

#グリッドリサーチの実施
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorithm, params, cv=stratifiedKfold)
gs.fit(x_train, y_train)

#ベストなモデルを取得して検証データの分類
best = gs.best_estimator_
best_pred = best.predict(x_test)
print(best)


SVC(C=1000, gamma=1e-05, random_state=123)


In [22]:
#上で作ったモデルの精度を確認する
score = best.score(x_test, y_test)
print(f'スコア: {score:.4f}')

#混同行列を出力
from sklearn.metrics import confusion_matrix
#空白
print()
print('混同行列')
print(confusion_matrix(y_test, best_pred))

スコア: 0.9825

混同行列
[[20  1]
 [ 0 36]]
